In [ ]:
#Jake Hu, UIN 658882246
#Project 1 FIN 553
import tensorflow as tf

In [ ]:
Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 6   # Polynmial order
Δt = T / m  # interval between two exercise dates

In [ ]:
def chebyshev_basis(x, k):
    B = tf.Variable(tf.ones([k, len(x)]))
    tf.compat.v1.assign(B[1,:],x)
    for n in range(2, k):
        tf.compat.v1.assign(B[n, :],(2 * x * B[n - 1, :] - B[n - 2, :]))

    return tf.transpose(B)

In [ ]:
def scale(x):
        xmin = tf.reduce_min(x)
        xmax = tf.reduce_max(x)
        a = 2 / (xmax - xmin)
        b = 1 - a * xmax
        return a * x + b

In [ ]:
def scale(x):
        xmin = tf.reduce_min(x)
        xmax = tf.reduce_max(x)
        a = 2 / (xmax - xmin)
        b = 1 - a * xmax
        return a * x + b

In [ ]:
def advance(S, r, σ, Δt, n):
    dB = tf.sqrt(Δt)*tf.Variable(tf.random.normal([n]))
    out = S + r * S * Δt + σ * S * dB
    return out

In [ ]:
def first_one(x):
    original = x
    x = tf.cast(tf.greater(x, 0.),dtype=tf.float32)
    nt = x.shape[0]
    batch_size = x.shape[1]
    x_not = 1 - x
    sum_x = tf.math.minimum(tf.math.cumprod(x_not, axis=0), 1.)
    ones = tf.ones([1, batch_size])
    lag = sum_x[:(nt - 1), :]
    lag = tf.concat([ones, lag],axis=0)
    return original * (lag * x)

In [ ]:
def compute_price(order, Spot, σ, K, r):
    tf.random.set_seed(11)
    S = tf.Variable(tf.zeros([m + 1, n]))
    tf.compat.v1.assign(S[0, :],tf.ones([n])*Spot)

    for t in range(m):
        tf.compat.v1.assign(S[t + 1, :], advance(S[t, :], r, σ, Δt, n))

    discount = tf.exp(-r * Δt)
    CFL = tf.math.maximum(0., K - S)
    value = tf.Variable(tf.zeros((m, n)))
    tf.compat.v1.assign(value[-1],CFL[-1] * discount)
    CV = tf.Variable(tf.zeros((m, n)))

    for k in range(2, m + 1):
        t = m - k
        t_next = t + 1

        X = chebyshev_basis(scale(S[t_next]), order)
        Y = value[t_next]

        # regression to estimate the continuation value
        Θ = tf.linalg.solve(tf.transpose(X) @ X, tf.transpose(X) @ tf.transpose(tf.reshape(Y,[1,n])))
        tf.compat.v1.assign(CV[t], tf.reshape(X @ Θ,[n]))
        tf.compat.v1.assign(value[t], discount * tf.where(CFL[t_next] > CV[t],CFL[t_next],value[t_next]))                                                                            
    POF = tf.where(CV<CFL[1:],CFL[1:],0.)
    FPOF = first_one(POF)
    m_range = tf.reshape(tf.Variable(range(m),dtype=tf.float32),[-1, 1])
    dFPOF = FPOF * tf.exp(-r * m_range * Δt)
    PRICE = tf.reduce_sum(dFPOF)/ n
    return PRICE

In [ ]:
print(compute_price(order, Spot, σ, K, r))

tf.Tensor(4.4784684, shape=(), dtype=float32)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# this function is used to unscale prediction from neural network
# we scale Y in order to speed up learning process
# therefore we need to unscale to get right prediction
def Unscale(Y,prediction):
        ymin = tf.reduce_min(Y)
        ymax = tf.reduce_max(Y)
        a = 2 / (ymax - ymin)
        b = 1 - a * ymax
        return (prediction-b)/a
        

In [ ]:
#define neural network
def nn(x,y):
        model = Sequential([Dense(64, activation='relu'), Dense(1)])
        opt = tf.keras.optimizers.SGD(learning_rate=0.1)
        model.compile(loss='mse', optimizer=opt)
        model.fit(tf.reshape(x, shape=[-1, 1]), y, epochs=12, batch_size=int(100000/100),verbose=False)
        prediction = model.predict(tf.reshape(x, shape=[-1, 1]))
        return prediction

In [ ]:
def compute_price_neuralnetwork(order, Spot, σ, K, r):
    tf.random.set_seed(11)
    S = tf.Variable(tf.zeros([m + 1, n]))
    tf.compat.v1.assign(S[0, :],tf.ones([n])*Spot)

    for t in range(m):
        tf.compat.v1.assign(S[t + 1, :], advance(S[t, :], r, σ, Δt, n))

    discount = tf.exp(-r * Δt)
    CFL = tf.math.maximum(0., K - S)
    value = tf.Variable(tf.zeros((m, n)))
    tf.compat.v1.assign(value[-1],CFL[-1] * discount)
    CV = tf.Variable(tf.zeros((m, n)))

    for k in range(2, m + 1):
        t = m - k
        t_next = t + 1

        X = scale(S[t_next])
        Y = scale(value[t_next])       

        # neural netwrok to estimate the continuation value


        prediction = Unscale(value[t_next],nn(tf.reshape(X, shape=[-1, 1]),Y))
        tf.compat.v1.assign(CV[t], tf.reshape(prediction,[n]))
        tf.compat.v1.assign(value[t], discount * tf.where(CFL[t_next] > CV[t],CFL[t_next],value[t_next]))                                                                            
    POF = tf.where(CV<CFL[1:],CFL[1:],0.)
    FPOF = first_one(POF)
    m_range = tf.reshape(tf.Variable(range(m),dtype=tf.float32),[-1, 1])
    dFPOF = FPOF * tf.exp(-r * m_range * Δt)
    PRICE = tf.reduce_sum(dFPOF)/ n
    return PRICE

In [ ]:
print(compute_price_neuralnetwork(order, Spot, σ, K, r))

tf.Tensor(4.470001, shape=(), dtype=float32)
